In [1]:
import pandas as pd
from datetime import datetime, timedelta
from pyshark.packet.fields import LayerFieldsContainer
import pyshark
import os
import gc

from anubisflow import AnubisFG

In [2]:
pd.set_option('display.max_columns', 200)

## NTP

In [3]:
data = pd.read_csv('../data/raw/csv/01-12/DrDoS_NTP.csv')
data.columns = [x.strip() for x in data.columns]
print(data.shape)
data.head()

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "/home/caiolmart/.local/lib/python3.8/site-packages/IPython/core/interactiveshell.py", line 3331, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-3-896377296c37>", line 1, in <module>
    data = pd.read_csv('../data/raw/csv/01-12/DrDoS_NTP.csv')
  File "/home/caiolmart/.local/lib/python3.8/site-packages/pandas/io/parsers.py", line 676, in parser_f
    return _read(filepath_or_buffer, kwds)
  File "/home/caiolmart/.local/lib/python3.8/site-packages/pandas/io/parsers.py", line 454, in _read
    data = parser.read(nrows)
  File "/home/caiolmart/.local/lib/python3.8/site-packages/pandas/io/parsers.py", line 1133, in read
    ret = self._engine.read(nrows)
  File "/home/caiolmart/.local/lib/python3.8/site-packages/pandas/io/parsers.py", line 2037, in read
    data = self._reader.read(nrows)
  File "pandas/_libs/parsers.pyx", line 860, in pandas._libs.parsers.TextReader.read
  File "pandas/_libs/parsers.pyx", 

KeyboardInterrupt: 

In [ ]:
data['Timestamp'][0]

In [ ]:
data['timestamp'] = data['Timestamp'].apply(lambda x: datetime.strptime(x, '%Y-%m-%d %H:%M:%S.%f') + timedelta(hours=4))
data.head()

In [ ]:
data['lst_timestamp'] = data.apply(lambda x: x['timestamp'] + timedelta(microseconds=x['Flow Duration']), axis=1)
data.head()

In [ ]:
type(data['Source Port'][0])

In [ ]:
type(data['Protocol'][0])

In [ ]:
protocols = {
    0 : 'HOPOPT',
    1 : 'ICMP',
    2 : 'IGMP',
    3 : 'GGP',
    4 : 'IPv4',
    5 : 'ST',
    6 : 'TCP',
    7 : 'CBT',
    8 : 'EGP',
    9 : 'IGP',
    10 : 'BBN-RCC-MON',
    11 : 'NVP-II',
    12 : 'PUP',
    13 : 'ARGUS',
    14 : 'EMCON',
    15 : 'XNET',
    16 : 'CHAOS',
    17 : 'UDP',
}

In [ ]:
data['flow_key'] = data.apply(lambda x: (LayerFieldsContainer(x['Source IP']),
                                         LayerFieldsContainer(x['Source Port']),
                                         LayerFieldsContainer(x['Destination IP']),
                                         LayerFieldsContainer(x['Destination Port']),
                                         protocols[x['Protocol']]), axis=1)
data.head()

In [11]:
flows = data[['Label', 'lst_timestamp', 'flow_key']]
flows.columns = [x.lower() for x in flows.columns]
flows = flows.sort_values('lst_timestamp')

flows.head()

,label,lst_timestamp,flow_key
2,BENIGN,2018-12-01 09:17:12.634572,"(65.55.163.78, 443, 192.168.50.7, 50458, TCP)"
3,BENIGN,2018-12-01 09:17:13.458373,"(65.55.163.78, 443, 192.168.50.7, 50465, TCP)"
0,DrDoS_NTP,2018-12-01 09:17:16.404686,"(172.16.0.5, 60675, 192.168.50.1, 80, TCP)"
10,DrDoS_NTP,2018-12-01 09:17:16.404865,"(192.168.50.1, 80, 172.16.0.5, 60675, TCP)"
11,DrDoS_NTP,2018-12-01 09:17:16.404963,"(172.16.0.5, 60675, 192.168.50.1, 80, TCP)"


In [20]:
flows.iloc[0, 1].to_pydatetime()

datetime.datetime(2018, 12, 1, 9, 17, 12, 634572)

In [25]:
flows.flow_key[0]

('172.16.0.5', '60675', '192.168.50.1', '80', 'TCP')

In [ ]:
packet

In [43]:
data.tail()

,Unnamed: 0,Flow ID,Source IP,Source Port,Destination IP,Destination Port,Protocol,Timestamp,Flow Duration,Total Fwd Packets,Total Backward Packets,Total Length of Fwd Packets,Total Length of Bwd Packets,Fwd Packet Length Max,Fwd Packet Length Min,Fwd Packet Length Mean,Fwd Packet Length Std,Bwd Packet Length Max,Bwd Packet Length Min,Bwd Packet Length Mean,Bwd Packet Length Std,Flow Bytes/s,Flow Packets/s,Flow IAT Mean,Flow IAT Std,Flow IAT Max,Flow IAT Min,Fwd IAT Total,Fwd IAT Mean,Fwd IAT Std,Fwd IAT Max,Fwd IAT Min,Bwd IAT Total,Bwd IAT Mean,Bwd IAT Std,Bwd IAT Max,Bwd IAT Min,Fwd PSH Flags,Bwd PSH Flags,Fwd URG Flags,Bwd URG Flags,Fwd Header Length,Bwd Header Length,Fwd Packets/s,Bwd Packets/s,Min Packet Length,Max Packet Length,Packet Length Mean,Packet Length Std,Packet Length Variance,FIN Flag Count,SYN Flag Count,RST Flag Count,PSH Flag Count,ACK Flag Count,URG Flag Count,CWE Flag Count,ECE Flag Count,Down/Up Ratio,Average Packet Size,Avg Fwd Segment Size,Avg Bwd Segment Size,Fwd Header Length.1,Fwd Avg Bytes/Bulk,Fwd Avg Packets/Bulk,Fwd Avg Bulk Rate,Bwd Avg Bytes/Bulk,Bwd Avg Packets/Bulk,Bwd Avg Bulk Rate,Subflow Fwd Packets,Subflow Fwd Bytes,Subflow Bwd Packets,Subflow Bwd Bytes,Init_Win_bytes_forward,Init_Win_bytes_backward,act_data_pkt_fwd,min_seg_size_forward,Active Mean,Active Std,Active Max,Active Min,Idle Mean,Idle Std,Idle Max,Idle Min,SimillarHTTP,Inbound,Label,timestamp,lst_timestamp,flow_key
1217002,2414,172.16.0.5-192.168.50.1-634-63125-17,172.16.0.5,634,192.168.50.1,63125,17,2018-12-01 10:51:39.600556,37146,200,0,88000.0,0.0,440.0,440.0,440.0,0.0,0.0,0.0,0.0,0.0,2.369030e+06,5384.159802,186.663317,264.430247,1961.0,1.0,37146.0,186.663317,264.430247,1961.0,1.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0,4000,0,5384.159802,0.0,440.0,440.0,440.0,0.0,0.0,0,0,0,0,0,0,0,0,0.0,442.200000,440.0,0.0,4000,0,0,0,0,0,0,200,88000,0,0,-1,-1,199,20,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,1,DrDoS_NTP,2018-12-01 10:51:39.600556,2018-12-01 10:51:39.637702,"(172.16.0.5, 634, 192.168.50.1, 63125, UDP)"
1217003,1506,172.16.0.5-192.168.50.1-634-12431-17,172.16.0.5,634,192.168.50.1,12431,17,2018-12-01 10:51:39.635018,42011,200,0,88000.0,0.0,440.0,440.0,440.0,0.0,0.0,0.0,0.0,0.0,2.094689e+06,4760.657923,211.110553,356.905935,3453.0,0.0,42011.0,211.110553,356.905935,3453.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0,4000,0,4760.657923,0.0,440.0,440.0,440.0,0.0,0.0,0,0,0,0,0,0,0,0,0.0,442.200000,440.0,0.0,4000,0,0,0,0,0,0,200,88000,0,0,-1,-1,199,20,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,1,DrDoS_NTP,2018-12-01 10:51:39.635018,2018-12-01 10:51:39.677029,"(172.16.0.5, 634, 192.168.50.1, 12431, UDP)"
1217004,195,172.16.0.5-192.168.50.1-634-881-17,172.16.0.5,634,192.168.50.1,881,17,2018-12-01 10:51:39.685553,40375,200,0,88000.0,0.0,440.0,440.0,440.0,0.0,0.0,0.0,0.0,0.0,2.179567e+06,4953.560372,202.889447,277.745829,1655.0,1.0,40375.0,202.889447,277.745829,1655.0,1.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0,4000,0,4953.560372,0.0,440.0,440.0,440.0,0.0,0.0,0,0,0,0,0,0,0,0,0.0,442.200000,440.0,0.0,4000,0,0,0,0,0,0,200,88000,0,0,-1,-1,199,20,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,1,DrDoS_NTP,2018-12-01 10:51:39.685553,2018-12-01 10:51:39.725928,"(172.16.0.5, 634, 192.168.50.1, 881, UDP)"
1217005,266,172.16.0.5-192.168.50.1-634-7051-17,172.16.0.5,634,192.168.50.1,7051,17,2018-12-01 10:51:39.721301,59116,200,0,88000.0,0.0,440.0,440.0,440.0,0.0,0.0,0.0,0.0,0.0,1.488599e+06,3383.178835,297.065327,625.349624,6203.0,1.0,59116.0,297.065327,625.349624,6203.0,1.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0,4000,0,3383.178835,0.0,440.0,440.0,440.0,0.0,0.0,0,0,0,0,0,0,0,0,0.0,442.200000,440.0,0.0,4000,0,0,0,0,0,0,200,88000,0,0,-1,-1,199,20,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,1,DrDoS_NTP,2018-12-01 10:51:39.721301,2018-12-01 10:51:39.780417,"(172.16.0.5, 634, 192.168.50.1, 7051, UDP)"
1217006,508,172.16.0.5-192.168.50.1-634-60495-17,172.16.0.5,634,192.168.50.1,60495,17,2018-12-01 10:51:39.778786,34660,103,0,45320.0,0.0,440.0,440.0,440.0,0.0,0.0,0.0,0.0,0.0,1.307559e+06,2971.725332,339.803922,788.010278,5653.0,1.0,34660.

## First and timestamp of files

In [4]:
files = os.listdir('../data/raw/csv/01-12')

In [6]:
time_dict = dict()
for file in files:
    row = pd.read_csv(f'../data/raw/csv/01-12/{file}')
    row.columns = [x.strip() for x in row.columns]
    tsi = row.Timestamp.iloc[0]
    tsf = row.Timestamp.iloc[-1]
    time_dict[file] = (tsi, tsf)
    print(f'File {file} goes from {tsi} to {tsf}')
del row
gc.collect()

File DrDoS_MSSQL.csv goes from 2018-12-01 11:32:32.915441 to 2018-12-01 11:47:08.463107
File UDPLag.csv goes from 2018-12-01 13:04:45.928673 to 2018-12-01 13:30:30.740425


ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "/home/caiolmart/.local/lib/python3.8/site-packages/IPython/core/interactiveshell.py", line 3331, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-6-f7f37c46c0e8>", line 3, in <module>
    row = pd.read_csv(f'../data/raw/csv/01-12/{file}')
  File "/home/caiolmart/.local/lib/python3.8/site-packages/pandas/io/parsers.py", line 676, in parser_f
    return _read(filepath_or_buffer, kwds)
  File "/home/caiolmart/.local/lib/python3.8/site-packages/pandas/io/parsers.py", line 454, in _read
    data = parser.read(nrows)
  File "/home/caiolmart/.local/lib/python3.8/site-packages/pandas/io/parsers.py", line 1148, in read
    df = DataFrame(col_dict, columns=columns, index=index)
  File "/home/caiolmart/.local/lib/python3.8/site-packages/pandas/core/frame.py", line 435, in __init__
    mgr = init_dict(data, index, columns, dtype=dtype)
  File "/home/caiolmart/.local/lib/python3.8/site-packages/pandas/core/internals/

KeyboardInterrupt: 

In [ ]:
sorted(time_dict.items(), key=lambda x: x[1][0])

In [54]:
sorted(time_dict.items(), key=lambda x: x[1])

[('DrDoS_NTP.csv', '2018-12-01 09:17:11.183810'),
 ('DrDoS_DNS.csv', '2018-12-01 10:51:39.813448'),
 ('DrDoS_LDAP.csv', '2018-12-01 11:22:40.254769'),
 ('DrDoS_MSSQL.csv', '2018-12-01 11:32:32.915441'),
 ('DrDoS_NetBIOS.csv', '2018-12-01 11:47:08.463789'),
 ('DrDoS_SNMP.csv', '2018-12-01 12:00:13.902782'),
 ('DrDoS_SSDP.csv', '2018-12-01 12:23:13.663425'),
 ('DrDoS_UDP.csv', '2018-12-01 12:36:57.628026'),
 ('UDPLag.csv', '2018-12-01 13:04:45.928673'),
 ('Syn.csv', '2018-12-01 13:30:30.741451'),
 ('TFTP.csv', '2018-12-01 13:34:27.403713')]

In [44]:
row = pd.read_csv(f'../data/raw/csv/01-12/DrDoS_DNS.csv', nrows=1)
row

,Unnamed: 0,Flow ID,Source IP,Source Port,Destination IP,Destination Port,Protocol,Timestamp,Flow Duration,Total Fwd Packets,Total Backward Packets,Total Length of Fwd Packets,Total Length of Bwd Packets,Fwd Packet Length Max,Fwd Packet Length Min,Fwd Packet Length Mean,Fwd Packet Length Std,Bwd Packet Length Max,Bwd Packet Length Min,Bwd Packet Length Mean,Bwd Packet Length Std,Flow Bytes/s,Flow Packets/s,Flow IAT Mean,Flow IAT Std,Flow IAT Max,Flow IAT Min,Fwd IAT Total,Fwd IAT Mean,Fwd IAT Std,Fwd IAT Max,Fwd IAT Min,Bwd IAT Total,Bwd IAT Mean,Bwd IAT Std,Bwd IAT Max,Bwd IAT Min,Fwd PSH Flags,Bwd PSH Flags,Fwd URG Flags,Bwd URG Flags,Fwd Header Length,Bwd Header Length,Fwd Packets/s,Bwd Packets/s,Min Packet Length,Max Packet Length,Packet Length Mean,Packet Length Std,Packet Length Variance,FIN Flag Count,SYN Flag Count,RST Flag Count,PSH Flag Count,ACK Flag Count,URG Flag Count,CWE Flag Count,ECE Flag Count,Down/Up Ratio,Average Packet Size,Avg Fwd Segment Size,Avg Bwd Segment Size,Fwd Header Length.1,Fwd Avg Bytes/Bulk,Fwd Avg Packets/Bulk,Fwd Avg Bulk Rate,Bwd Avg Bytes/Bulk,Bwd Avg Packets/Bulk,Bwd Avg Bulk Rate,Subflow Fwd Packets,Subflow Fwd Bytes,Subflow Bwd Packets,Subflow Bwd Bytes,Init_Win_bytes_forward,Init_Win_bytes_backward,act_data_pkt_fwd,min_seg_size_forward,Active Mean,Active Std,Active Max,Active Min,Idle Mean,Idle Std,Idle Max,Idle Min,SimillarHTTP,Inbound,Label
0,425,172.16.0.5-192.168.50.1-634-60495-17,172.16.0.5,634,192.168.50.1,60495,17,2018-12-01 10:51:39.813448,28415,97,0,42680.0,0.0,440.0,440.0,440.0,0.0,0.0,0.0,0.0,0.0,1.502024e+06,3413.689952,295.989583,500.959301,3596.0,1.0,28415.0,295.989583,500.959301,3596.0,1.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0,-97,0,3413.689952,0.0,440.0,440.0,440.0,0.0,0.0,0,0,0,0,0,0,0,0,0.0,444.536082,440.0,0.0,-97,0,0,0,0,0,0,97,42680,0,0,-1,-1,96,-1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,1,DrDoS_DNS
